In [ ]:
# Import the required libraries and dependencies
import pandas as pd
from pathlib import Path
import sqlalchemy

## Import the data 

In [ ]:
# Import large kickstarter recent dataset
kickstarter_large_recent_df = pd.read_csv(
    Path('./Resources/kickstarter_data/ks-projects-201801.csv')
)

# Imported data has a space at the end of the column name
# Remove spaces in columns name
kickstarter_large_recent_df.columns = kickstarter_large_recent_df.columns.str.replace(' ','')

# Set the index as the ID
kickstarter_large_recent_df.set_index('ID', inplace=True)

# View head
kickstarter_large_recent_df.head(2)

In [ ]:
# Import small kickstarter most backed dataset
kickstarter_small_most_backed_df = pd.read_csv(
    Path('./Resources/kickstarter_data/most_backed.csv')
)

# Fix ID column
kickstarter_small_most_backed_df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

# Set the index as the ID
kickstarter_small_most_backed_df.set_index('ID', inplace=True)

# View head
kickstarter_small_most_backed_df.head(2)

In [ ]:
# Import indiegogo dataset
indiegogo_df = pd.read_csv(
    Path('./Resources/indiegogo_data/indiegogo.csv')
)

# Fix ID column
indiegogo_df.rename(columns={'project_id' : 'ID'}, inplace=True)

# Set the index as the ID
indiegogo_df.set_index('ID', inplace=True)

# View head
indiegogo_df.head(2)

## Rename cols

In [ ]:
# Rename the indiegogo columns to match the kickstarter df
indiegogo_df.rename(columns={'category' : 'main_category', 'title' : 'name', 'amount_raised_usd' : 'usd_pledged_real', 'goal_usd' : 'usd_goal_real'}, inplace=True)

kickstarter_small_most_backed_df.rename(columns={'amt.pledged' : 'usd_pledged_real', 'title' : 'name', 'category' : 'main_category', 'goal' : 'usd_goal_real', 'num.backers' : 'backers'}, inplace=True)
# View head
indiegogo_df.head(2)
kickstarter_small_most_backed_df.head(2)

## Clean large kickstarter df

In [ ]:
# Kickstarter code to fix dates
kickstarter_large_recent_df['launched'] = pd.to_datetime(kickstarter_large_recent_df['launched'])
kickstarter_large_recent_df['deadline'] = pd.to_datetime(kickstarter_large_recent_df['deadline'])

# Only pull out failed and successful cases to match indiegogo df
failure_success = ['failed','successful']

# Quick check to see if we have already renamed state col
if 'failed' in kickstarter_large_recent_df['state'].values:
    kickstarter_large_recent_df = kickstarter_large_recent_df[kickstarter_large_recent_df['state'].isin(failure_success)]
    # Rename successful -> 1 and failed -> 0 to match indiegogo
    kickstarter_large_recent_df['state'].replace({'failed': '0', 'successful': '1'}, inplace=True)
    # Fix types
    kickstarter_large_recent_df = kickstarter_large_recent_df.astype({'state': 'int64'})

# Sometimes projects are successful with no backers. Remove this data
kickstarter_large_recent_df.drop(kickstarter_large_recent_df[(kickstarter_large_recent_df['state'] == 1) & (kickstarter_large_recent_df['backers'] == 0)].index, inplace = True)

# Remove all countries with that have euro currency and country equal to N,0"
kickstarter_large_recent_df.drop(kickstarter_large_recent_df[(kickstarter_large_recent_df['currency'] == 'EUR') & (kickstarter_large_recent_df['country'] == 'N,0"')].index, inplace = True)

# Create a dictionary of country and currency pairs to fix other N,0" countries to right country
country_currency_df = kickstarter_large_recent_df.loc[:,['country', 'currency']]
country_currency_df.drop(kickstarter_large_recent_df[kickstarter_large_recent_df['country'] == 'N,0"'].index, inplace=True)
country_currency_df.drop_duplicates(inplace=True)
country_currency_df.set_index('currency', inplace = True)
currency_country_dict = country_currency_df.to_dict()['country']

# Function to clean up country col
def replace_N0(country, currency):
    if country == 'N,0"':
        return currency_country_dict[currency]
    else:
        return country

# Clean up all the N,0" values for countries using the currency_country_dict
kickstarter_large_recent_df['country'] = kickstarter_large_recent_df.apply(lambda row: replace_N0(row['country'], row['currency']), axis=1)

## Clean indiegogo df

In [ ]:
# Indiegogo code to fix dates
indiegogo_df['launched'] = pd.to_datetime(indiegogo_df['date_launch'] + ' ' + indiegogo_df['time_launch'])
indiegogo_df['deadline'] = pd.to_datetime(indiegogo_df['date_end'] + ' ' + indiegogo_df['time_end'])

## Clean small kickstarter df

In [ ]:
# Only look at US
kickstarter_small_most_backed_df = kickstarter_small_most_backed_df[kickstarter_small_most_backed_df['currency'] == 'usd']

## Create dfs from selected columns

In [ ]:
# List of kickstart and indiegogo columns to drop
kickstarter_cols_drop = ['category', 'goal', 'pledged', 'usdpledged']
kickstarter_small_cols_drop = ['blurb', 'by', 'url']
indiegogo_cols_drop = ['year_end',
'month_end',
'day_end',
'amount_raised',
'in_demand',
'year_launch',
'month_launch',
'day_launch',
'tagline',
'url',
'australia',
'canada',
'switzerland',
'denmark',
'western_europe',
'great_britain',
'hong_kong',
'norway',
'sweden',
'singapore',
'united_states',
'education',
'productivity',
'energy_greentech',
'wellness',
'comics',
'fashion_wearables',
'video_games',
'photography',
'tv_shows',
'dance_theater',
'phones_accessories',
'audio',
'film',
'transportation',
'art',
'environment',
'writing_publishing',
'music',
'travel_outdoors',
'health_fitness',
'tabletop_games',
'home',
'local_business',
'food_beverage',
'culture',
'human_rights',
'podcasts_vlogs',
'camera_gear',
'time_end',
'time_launch',
'date_launch',
'date_end',
'jan',
'feb',
'mar',
'apr',
'may',
'jun',
'jul',
'aug',
'sep',
'oct',
'nov',
'dec',
'tperiod']

# Selected columns kickstarter df
kickstarter_selected_cols_large_recent_df = kickstarter_large_recent_df.drop(kickstarter_cols_drop, axis=1)

# Selected columns kickstarter small df
kickstarter_selected_cols_small_most_backed_df = kickstarter_small_most_backed_df.drop(kickstarter_small_cols_drop, axis=1)

# Selected columns indiegogo df
indiegogo_selected_cols_df = indiegogo_df.drop(indiegogo_cols_drop, axis=1)

## Add new kickstarter cols

In [ ]:
# Duration
kickstarter_selected_cols_large_recent_df['duration'] = kickstarter_selected_cols_large_recent_df['deadline'] - kickstarter_selected_cols_large_recent_df['launched'] 
kickstarter_selected_cols_large_recent_df['duration'] = kickstarter_selected_cols_large_recent_df['duration'].dt.days

# Daily Goal 
kickstarter_selected_cols_large_recent_df['daily_goal'] = round(kickstarter_selected_cols_large_recent_df['usd_goal_real'] / kickstarter_selected_cols_large_recent_df['duration'],2)

# Daily Pledged
kickstarter_selected_cols_large_recent_df['daily_pledged'] = round(kickstarter_selected_cols_large_recent_df['usd_pledged_real'] / kickstarter_selected_cols_large_recent_df['duration'],2)

# Funded Percentage
kickstarter_selected_cols_large_recent_df['funded_percent'] = round(kickstarter_selected_cols_large_recent_df['usd_pledged_real'] / kickstarter_selected_cols_large_recent_df['usd_goal_real'],4)

# Average Backer Per Day
kickstarter_selected_cols_large_recent_df['avg_backer_per_day'] = round(kickstarter_selected_cols_large_recent_df['backers'] / kickstarter_selected_cols_large_recent_df['duration'],2)

# Pledged Per Person
kickstarter_selected_cols_large_recent_df['pledged_per_person'] = round(kickstarter_selected_cols_large_recent_df['usd_pledged_real'] / kickstarter_selected_cols_large_recent_df['backers'],2)
kickstarter_selected_cols_large_recent_df['pledged_per_person'] = kickstarter_selected_cols_large_recent_df['pledged_per_person'].fillna(0)

# Fix types
kickstarter_selected_cols_large_recent_df = kickstarter_selected_cols_large_recent_df.astype({"funded_percent": 'float', "avg_backer_per_day":'float', "pledged_per_person": 'float'})

## Add new indiegogo cols

In [ ]:
# Duration
indiegogo_selected_cols_df['duration'] = indiegogo_selected_cols_df['deadline'] - indiegogo_selected_cols_df['launched'] 
indiegogo_selected_cols_df['duration'] = indiegogo_selected_cols_df['duration'].dt.days

# Daily Goal 
indiegogo_selected_cols_df['daily_goal'] = round(indiegogo_selected_cols_df['usd_goal_real'] / indiegogo_selected_cols_df['duration'],2)

# Daily Pledged
indiegogo_selected_cols_df['daily_pledged'] = round(indiegogo_selected_cols_df['usd_pledged_real'] / indiegogo_selected_cols_df['duration'],2)

# Funded Percentage
indiegogo_selected_cols_df['funded_percent'] = round(indiegogo_selected_cols_df['usd_pledged_real'] / indiegogo_selected_cols_df['usd_goal_real'],4)

# Fix types
indiegogo_selected_cols_df = indiegogo_selected_cols_df.astype({"funded_percent": 'float'})

## Reorder cols

In [ ]:
# Base column names for reordering
base_order = ['name', 'main_category', 'currency', 'usd_goal_real', 'usd_pledged_real', 'deadline', 'launched', 'state', 'funded_percent','duration', 'daily_goal', 'daily_pledged']
small_kickstarter_order = ['name', 'main_category', 'currency', 'usd_goal_real', 'usd_pledged_real', 'location', 'backers', 'num.backers.tier', 'pledge.tier']
kickstarter_order = base_order + ['country', 'avg_backer_per_day', 'pledged_per_person']
indiegogo_order = base_order

# Reorder each df
kickstarter_selected_cols_large_recent_df = kickstarter_selected_cols_large_recent_df[kickstarter_order]
kickstarter_selected_cols_small_most_backed_df = kickstarter_selected_cols_small_most_backed_df[small_kickstarter_order]
indiegogo_selected_cols_df = indiegogo_selected_cols_df[indiegogo_order]

## Write clean data to files

In [ ]:
kickstarter_selected_cols_large_recent_df.to_csv('./Resources/kickstarter_data_clean/ks-projects-large.csv')
kickstarter_selected_cols_small_most_backed_df.to_csv('./Resources/kickstarter_data_clean/ks-projects-small.csv')
indiegogo_selected_cols_df.to_csv('./Resources/kickstarter_data_clean/indiegogo-projects.csv')